# Implementação de Métodos de Machine Learning
**Aluno:** Matheus Gama dos Santos - 20180163117<br>
**Curso:** Ciência da Computação - UFPB<br>
**Profª:** Thais Gaudencio do Rego<br><br>
Este projeto consiste na implementação de métodos de Machine Learning para analisar as bases de dados listadas abaixo:
- [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data)
- [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)
- [2018-2019 Premier League Data
](https://www.kaggle.com/thesiff/premierleague1819)

Cada base de dados será tratada em uma das 3 seções: **Classificação**, **Regressão** e **Clusterização** (nessa ordem).<br>

## Classificação

In [ ]:
#install imblearn
!pip install imblearn

In [ ]:
#import modules
import pandas as pd
import numpy as np
from sklearn import *
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.metrics import sensitivity_score, specificity_score
from string import ascii_letters
import ipywidgets as widgets

In [ ]:
#load dataframe
titanic_filename = 'titanic/train.csv'
titanic_dataframe = pd.read_csv(titanic_filename)

In [ ]:
titanic_dataframe.head()

### Preprocessing

In [ ]:
# select useful attributes
titanic_dataframe = titanic_dataframe.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Fare"], axis=1)
titanic_dataframe.head()

In [ ]:
# converting "Sex" attribute
titanic_dataframe.Sex = titanic_dataframe.Sex.replace({'male':0, 'female':1})
titanic_dataframe.head()

In [ ]:
titanic_dataframe.describe()

<font size="3">&emsp;&emsp;Analisando o estado atual do *Dataframe*, observa-se que o atributo *Age* possui mais de 100 linhas sem dado. Vamos realizar testes para verificar a influência das linhas com dado faltando.</font><br><br>

In [ ]:
def get_titanic_survivability(d_frame_surv): # d_frame_surv: dataframe Survived column
    zeros = 0
    ones = 0
    for numbers in d_frame_surv:
        if numbers:
            ones += 1
        else:
            zeros += 1
    print(f"Died: {zeros} ({zeros/(zeros+ones)}), Survived: {ones} ({ones/(zeros+ones)})")

In [ ]:
# Compare dataframe with and without missing data rows

# m_df will be titanic_dataframe.Survived without missing values
m_df = titanic_dataframe.dropna(axis=0)
m_df = np.array(m_df.Survived)

# dtf will be a copy of titanic_dataframe.Survived
df = np.array(titanic_dataframe.Survived)

In [ ]:
# Compare results
print('Without missing values:')
get_titanic_survivability(m_df)
print('\nWith missing values:')
get_titanic_survivability(df)

<font size="3">&emsp;&emsp;É possível observar que o balanceamento da base de dados, com relação ao *target* ( *Survived* ), não teve alteração significativa ao eliminar os objetos sem o valor do atributo *Age*. Vamos comparar, por fim, o impacto da mudança nos outros atributos.</font><br><br>

In [ ]:
# description of titanic_dataframe
titanic_dataframe.describe()

In [ ]:
# description of titanic_dataframe without the missing values
titanic_dataframe.dropna(axis=0).describe()

<font size="3">&emsp;&emsp;O impacto não se mostrou significativo na distribuição dos outros atributos. Agora, vamos normalizar os dados e exibir a matriz de correlação dos atributos relevantes.</font><br><br>

In [ ]:
titanic_dataframe = titanic_dataframe.dropna(axis=0)

# Normalizing dataframe
def normalize_df(df):
    d_values = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    d_values_scaled = min_max_scaler.fit_transform(d_values)
    df_norm = pd.DataFrame(d_values_scaled, columns=df.columns)
    return df_norm

titanic_dataframe_norm = normalize_df(titanic_dataframe)
titanic_dataframe_norm.sample(5)

In [ ]:
# Correlation Matrix
sns.heatmap(titanic_dataframe_norm.corr(), annot=True, fmt=".2f")
plt.show()

<font size="3">&emsp;&emsp;Em relação ao *target* (*Survived* ), é possível perceber que as correlações mais importantes são entre a classe do indivíduo ( -0,36 ) e o sexo ( 0,54 ). Quanto menor a classe social, mais chance de sobreviver. Também pode-se dizer que o sexo feminino tem mais chance de sobreviver do que o masculino. As outras correlações não apresentam valores expressivos. Analisaremos agora a presença de outliers em cada atributo.</font><br><br>

In [ ]:
titanic_dataframe.describe()

<font size="3">&emsp;&emsp;*Survived* é um atributo binário, que também é o target, portanto seus valores tem significado classificatório. Logo, survived não possui outliers. Pclass também possui valores com significado classificatório (1, 2 ou 3), logo não possui outliers. Sex é binário e também possui significado classificatório, logo não possui outliers. Age, aparentemente possui outliers, dado que sua média tem valor aproximado de 29,6991, com desvio padrão de aproximadamente 14,5264. Se considerarmos ( $média + (2dp)$ ), valores acima de 58,7519 podem ser considerados outliers. Porém, nesse caso, a base de dados possui valores entre 58,7519 e 80, que são valores representativos pra base de dados. Além do mais, talvez se utilizássemos ( $média + (3dp)$ ) não consideraríamos como outlier. O mesmo acontece com SibSp e Parch. Para verificar as afirmações, vamos observar os gráficos de cada atributo.</font><br><br>

In [ ]:
attribute = 'Survived'
def on_change(change):
    global attribute
    if change['type'] == 'change' and change['name'] == 'value':
        attribute = change['new']

In [ ]:
w = widgets.Dropdown(
    options=[e for e in titanic_dataframe.columns],
    value=titanic_dataframe.columns[0],
    description='Attribute:',
    disabled=False,
)

w.observe(on_change)

display(w)

##### Obs.: Selecione o atributo e execute as duas próximas células para exibir o histograma e boxplot

In [ ]:
# Plot histogram
titanic_dataframe[attribute].plot(kind='hist',title=attribute ,bins=50,figsize=(8,4))

In [ ]:
# Boxplot
titanic_dataframe[attribute].plot(kind='box',figsize=(8,4))

<font size="3">&emsp;&emsp;Ao observar os gráficos, vemos que SibSp e Parch são os atributos com verdadeiros outliers e não balanceados por essa mesma razão. Vamos retirar então os valores acima de ( $média + ( 3dp )$ ) dos dois atributos.</font><br><br>

In [ ]:
# Dropping outliers
indexNames = titanic_dataframe[ (titanic_dataframe['SibSp'] > 3) | (titanic_dataframe['Parch'] > 3) ].index
titanic_dataframe.drop(indexNames , inplace=True)

# get normalized version
titanic_dataframe_norm = normalize_df(titanic_dataframe)

<font size="3">Agora temos os dados balanceados e normalizados, prontos para aplicar os modelos.</font><br><br>

### Models

In [ ]:
# select features and target
y = titanic_dataframe_norm.Survived
x = titanic_dataframe_norm.drop(columns='Survived').to_numpy()

In [ ]:
# split train and test
train_X, test_X, train_Y, test_Y = model_selection.train_test_split( x, y, random_state=0, test_size=.2 )

<font size="3">&emsp;&emsp;Os modelos escolhidos foram Random Forest e SVM, com a razão da escolha de cada um descrita junto com os resultados após a execução.</font><br><br>

In [ ]:
# Random Forest
rdf = ensemble.RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
rdf.fit(train_X, train_Y)

In [ ]:
pred_Y = rdf.predict(test_X)
def show_metrics(test, pred):
    # accuracy: the fraction of predictions the model got right.
    accuracy = metrics.accuracy_score(test,pred)

    # sensitivy (recall): Percentage of correct predictions out of all the positive classes. (TP / TP+FN).
    sensitivity = sensitivity_score(test,pred)
    
    # specificity: proportion of actual negatives that are correctly identified as such. (TN / FP + TN).
    specificity = specificity_score(test,pred)
    
    """>Precision (used to calculate f1 score): Out of all the positive classes predicted correctly, how many are actually positive. (TP / TP+FP)"""
    # f1 score (harmonic mean of precision and recall): an accuracy coefficient with values between 0 and 1 (0 and 1 included).
    f1_score = metrics.f1_score(test, pred)

    x = np.array([accuracy, f1_score, sensitivity, specificity]).reshape(1,4)
    return pd.DataFrame(x, columns=["Accuracy", "F1 Score","Sensitivity", "Specificity"], index=["Results"])

results_r_forest = show_metrics(test_Y, pred_Y)
results_r_forest

In [ ]:
# confusion matrix
def show_confusion_mat(test,pred):
    c_mat = metrics.confusion_matrix(test, pred)
    return pd.DataFrame(c_mat, columns=["P", "N"], index=["P", "N"])
mat_r_forest = show_confusion_mat(test_Y,pred_Y)
mat_r_forest

# TP  FP
# FN  TN

<font size="3">&emsp;&emsp;Random Forest é um algoritmo de classificação e regressão, baseado em árvores de decisão, porém seu funcionamento corrige o hábito de realizar *overfitting* sobre a base de treinamento. Com acurácia de 0,861314 e F1-Score de 0,828829, o modelo se mostrou bem eficiente. Utilizando *max_depth* = 3, *random_state* = 2, obteve-se sensibilidade de 0,793103 e especificidade de 0,911392, que podem ser considerados bons resultados. Os valores para os parâmetros utilizados foram obtidos através de testes manuais e comparação.</font><br><br>

In [ ]:
# SVC (SVM)
SVC = svm.SVC(gamma=11)
SVC.fit(train_X, train_Y)

In [ ]:
pred_Y = SVC.predict(test_X)
results_SVC = show_metrics(test_Y, pred_Y)
results_SVC

In [ ]:
mat_SVC = show_confusion_mat(test_Y,pred_Y)
mat_SVC

<font size="3">&emsp;&emsp;SVM é classificador linear binário não probabilístico que tenta definir uma reta que melhor separa as classes do problema. O fato da razão entre as saídas ser próximo de 1 e os objetos da base só possuírem apenas duas classificações possíveis são a justificativa para a escolha desse algoritmo. Os resultados foram bem satisfatórios, com acurácia de 0,854015 e F1-Score de 0,821429. Os números de acertos positivos e negativos também encontrou altas taxas, com sensibilidade de 0,793103 e especificidade de 0,898734. Foram testados alguns valores para o parâmetro *gamma*, que foi o mais impactante nos resultados quando alterado, dentro do intervalo de 1 a 15, com melhor valor sendo *gamma* = 11. A mudança do gamma para outros valores ocasiona em troca de acurácia e especificidade por sensibilidade, ou apenas a perda em algum dos valores. O estado atual mantém sensibilidade e especificidade balanceados, diminuindo a chance de *overfitting*.</font><br><br>

## Regressão

In [ ]:
#load dataframes
hp_filename = 'house_prices/train.csv'
hp_dataframe = pd.read_csv(hp_filename, na_values=None,keep_default_na=False)

In [ ]:
hp_dataframe.describe()

### Preprocessing

In [ ]:
# converting non-numerical attributes
file = open('house_prices/dict.txt','r')
replaces = file.read()
replaces = replaces.split(";")
for each_element in replaces:
    attribute = each_element[2:each_element.find("'",2)]
    dictionary = each_element[each_element.find("{",2) +1: each_element.find("}")].split(', ')
    dict_rep = {}
    for each_n_element in dictionary:
        value = each_n_element[each_n_element.find(": ")+2:]
        dict_rep[each_n_element[1:each_n_element.find("'",2)]] = int(value)
    hp_dataframe[attribute].replace(dict_rep, inplace=True)
file.close()
hp_dataframe[hp_dataframe.select_dtypes("object").columns] = hp_dataframe[hp_dataframe.select_dtypes("object").columns].astype('int', inplace=True)

In [ ]:
# normalizing dataframe
hp_dataframe = normalize_df(hp_dataframe)
hp_dataframe.drop(columns=['Id'], inplace=True)
hp_dataframe.sample(5)

### Models

In [ ]:
# select features and target
y = hp_dataframe.SalePrice
x = hp_dataframe.drop(columns='SalePrice').to_numpy()
# split train and test
train_X, test_X, train_Y, test_Y = model_selection.train_test_split( x, y, random_state=0, test_size=.2 )

<font size="3">&emsp;&emsp;Os modelos escolhidos foram *Linear Regression* e *Random Forest Regression*. *Linear Regression* foi escolhido com o pensamento de que cada um dos atributos é indepente entre si, mas compõem a saída em conjunto. O motivo da escolha do *Random Forest* foi baseada na ideia de que *Decision Trees* tentam tomar decisões seguindo caminhos que melhor representem a situação, sendo a ideia parecida com o problema adotado (classificar o preço de uma casa baseado no preço de outras). *Random Forest* usa várias árvores de decisão e escolher a que melhor representa a ocasião, também com a vantagem de corrigir o hábito de *overfitting*, como já citado.</font>

#### Linear Regression

<font size="3">&emsp;&emsp;*Linear Regression* possui 4 parâmetros: fit_intercept, normalize, copy_X e n_jobs. Copy_x só faz com que a base de teste passada seja copiada para não ter risco de ser sobrescrita. Definit n_jobs para outro valor que não seja *None* ( valor *default* ) é inútil para casos onde temos apenas uma variável *target*. Fit_intercept e normalize são booleanos, sendo fit_intercept = True por padrão e normalize = False por padrão, sendo esse último ignorado se o parâmetro fit_intercept = False. Os resultados possíveis serão exibidos abaixo.</font><br><br>

In [ ]:
results = []

# fit_intercept = True,  normalize = False
lr = linear_model.LinearRegression().fit(train_X,train_Y)
lr_pred = lr.predict(test_X)

In [ ]:
# Mean Absolute Error
abs_error = metrics.mean_absolute_error(lr_pred,test_Y)
# Mean Squared Error
sqrt_error = metrics.mean_squared_error(lr_pred,test_Y)
results.append([abs_error, sqrt_error])

In [ ]:
# fit_intercept = False
lr = linear_model.LinearRegression(fit_intercept=False).fit(train_X,train_Y)
lr_pred = lr.predict(test_X)

In [ ]:
# Mean Absolute Error
abs_error = metrics.mean_absolute_error(lr_pred,test_Y)
# Mean Squared Error
sqrt_error = metrics.mean_squared_error(lr_pred,test_Y)
results.append([abs_error, sqrt_error])

In [ ]:
# normalize = True
lr = linear_model.LinearRegression(normalize=True).fit(train_X,train_Y)
lr_pred = lr.predict(test_X)

In [ ]:
# Mean Absolute Error
abs_error = metrics.mean_absolute_error(lr_pred,test_Y)
# Mean Squared Error
sqrt_error = metrics.mean_squared_error(lr_pred,test_Y)
results.append([abs_error, sqrt_error])

In [ ]:
report = pd.DataFrame(np.array(results),columns=['Mean Absolute Error', 'Mean Squared Error'], index=['default','fit_intercept = False','normalize = True'])
report

<font size="3">&emsp;&emsp;É observável que o menor erro médio absoluto é encontrado quando os parâmetros tem seus valores *default* (fit_intercept = True,  normalize = False). O valor do erro quadrático se encontra menor quando fit_intercept = False, provavelmente porque os dados não estão centralizados. Logo, temos como melhor caso a condição *default* (fit_intercept = True,  normalize = False), com *Mean Absolute Error* = 0,033362 e *Mean Squared Error* = 0,005529.</font><br><br>

#### Random Forest

<font size="3">&emsp;&emsp;O algoritmo Random Forest possui diversos parâmetros, porém iremos analisar o comportamento de 3 deles (por acreditar terem grande importância e por limitações computacionais): n_estimators, max_depth, random_state.</font><br><br>

In [ ]:
results = []

for n in range(500):
    rf_reg = ensemble.RandomForestRegressor(n_estimators=10, random_state=n).fit(train_X,train_Y)
    rf_pred = rf_reg.predict(test_X)
    # Mean Absolute Error
    abs_error = metrics.mean_absolute_error(rf_pred,test_Y)
    # Mean Squared Error
    sqrt_error = metrics.mean_squared_error(rf_pred,test_Y)
    results.append([abs_error, sqrt_error])

In [ ]:
r_abs = np.array([[n[0]] for n in results])
plt.plot([[n] for n in range(500)],r_abs)
plt.xlabel("n (random_state)")
plt.ylabel("Mean Absolute Error")
plt.title("Mean Absolute Error vs Random State")

In [ ]:
r_sqrt= np.array([[n[1]] for n in results])
plt.plot([[n] for n in range(500)],r_sqrt)
plt.xlabel("n (random_state)")
plt.ylabel("Mean Squared Error")
plt.title("Mean Squared Error vs Random State")

<font size="3">&emsp;&emsp;Random_state oferece melhores resultados nos 2 erros com valor aproximado de 370 a 390. O valor encontrado foi random_state=382. Vamos agora fazer o mesmo com n_estimators, fixando o random_state.</font><br><br>

In [ ]:
results = []
progress = 0.0
for n in range(1,401,7):
    rf_reg = ensemble.RandomForestRegressor(n_estimators=n, random_state=382).fit(train_X,train_Y)
    rf_pred = rf_reg.predict(test_X)
    # Mean Absolute Error
    abs_error = metrics.mean_absolute_error(rf_pred,test_Y)
    # Mean Squared Error
    sqrt_error = metrics.mean_squared_error(rf_pred,test_Y)
    results.append([abs_error, sqrt_error])
    progress += 1.0/58.0
    print(f'{progress*100}%', end='\r')

In [ ]:
r_abs = np.array([[n[0]] for n in results])
plt.plot([[n] for n in range(1,401,7)],r_abs)
plt.xlabel("n_estimators")
plt.ylabel("Mean Absolute Error")
plt.title("Mean Absolute Error vs Number of Estimators")

In [ ]:
r_sqrt= np.array([[n[1]] for n in results])
plt.plot([[n] for n in range(1,401,7)],r_sqrt)
plt.xlabel("n_estimators")
plt.ylabel("Mean Squared Error")
plt.title("Mean Squared Error vs Number of Estimators")

<font size="3">&emsp;&emsp;N_estimators oferece melhor resultado com valor aproximado entre 1 e 36, dado que os testes foram feitos de 7 em 7 devido à limitação de processamento, tomando como base o erro médio absoluto. O valor escolhido foi n_estimators=15. Vamos agora testar com alguns valores de max_depth.</font><br><br>

In [ ]:
results = []
progress = 0.0
for n in range(1,33):
    rf_reg = ensemble.RandomForestRegressor(n_estimators=15, random_state=382, max_depth=n).fit(train_X,train_Y)
    rf_pred = rf_reg.predict(test_X)
    # Mean Absolute Error
    abs_error = metrics.mean_absolute_error(rf_pred,test_Y)
    # Mean Squared Error
    sqrt_error = metrics.mean_squared_error(rf_pred,test_Y)
    results.append([abs_error, sqrt_error])
    progress += 1.0/32.0
    print(f'{(progress*100):.2f}%', end='\r')

In [ ]:
r_abs = np.array([[n[0]] for n in results])
plt.plot([[n] for n in range(1,33)],r_abs)
plt.xlabel("max_depth")
plt.ylabel("Mean Absolute Error")
plt.title("Mean Absolute Error vs Max Depth")

In [ ]:
r_sqrt= np.array([[n[1]] for n in results])
plt.plot([[n] for n in range(1,33)],r_sqrt)
plt.xlabel("max_depth")
plt.ylabel("Mean Squared Error")
plt.title("Mean Squared Error vs Max Depth")

<font size="3">&emsp;&emsp;Max_depth foi testado com valores até 40% do número de atributos, por garantia para a possibilidade de *overfitting* diminuir ainda mais. O valor ideal encontrado foi max_depth = 22, com resultado aproximado de *Mean Absolute Error* = 0.0240231 e *Mean Squared Error* = 0.00205843, ainda melhor que o *Linear Regression*.</font><br><br>

## Clusterização

In [ ]:
#load dataframe
premier_filename = 'premier_league/epl_1819.csv'
premier_dataframe = pd.read_csv(premier_filename)
premier_dataframe.head()

### Preprocessing

In [ ]:
# Drop Team
premier_dataframe.drop(columns=["Team"], inplace=True)

# replace nominal
premier_dataframe.category.replace({'Champions League': 0, 'Champions League Qualification': 1,
       'Europa League': 2, 'Europa League Qualification': 3,
       'No UEFA Competitions': 4, 'Relegated': 5}, inplace=True)

In [ ]:
# Turning all attributes into numbers
for elem in premier_dataframe.columns:
    if premier_dataframe[elem].dtype == 'O':
        for each_member in premier_dataframe[elem]:
            premier_dataframe[elem].replace(each_member, each_member.replace(',',''), inplace=True)

premier_dataframe[premier_dataframe.select_dtypes("object").columns] = premier_dataframe[premier_dataframe.select_dtypes("object").columns].astype('int', inplace=True)

### Models

In [ ]:
x = premier_dataframe.to_numpy()

In [ ]:
def print_labels(labels):
    for each_key in labels:
        print( f'{each_key}: {labels[each_key]}' )

#### K-Means

In [ ]:
labels_k = {2: [], 5: [], 10: []}
for each_case in [2,5,10]:
    km = cluster.KMeans(n_clusters=each_case, random_state=0).fit(x)
    labels_k[each_case] = km.labels_
print_labels(labels_k)

#### AgglomerativeClustering (Hierarchical)

In [ ]:
labels_a = {2: [], 5: [], 10: []}
for each_case in [2,5,10]:
    ag = cluster.AgglomerativeClustering(n_clusters=each_case).fit(x)
    labels_a[each_case] = ag.labels_
print_labels(labels_a)

<font size="3">Comparando os primeiros resultados:</font><br><br>

In [ ]:
for each_key in labels_k:
    labels_k[each_key] = labels_k[each_key].reshape(-1,1)
    labels_a[each_key] = labels_a[each_key].reshape(-1,1)
results = labels_k[2]
results = np.append(results,labels_a[2], axis=1)
results = np.append(results,labels_k[5], axis=1)
results = np.append(results,labels_a[5], axis=1)
results = np.append(results,labels_k[10], axis=1)
results = np.append(results,labels_a[10], axis=1)
results = pd.DataFrame( results, columns=['K-Means 2','Agglomerative 2','K-Means 5','Agglomerative 5','K-Means 10','Agglomerative 10'] )
results

<font size="3">&emsp;&emsp;Independente do valor de n_clusters, os algoritmos obtiveram os mesmos resultados. Os *labels* dos clusters não são os mesmos, mas são correspondentes, sendo os elementos agrupados da mesma forma. Vamos então fixar um número de clusters e alterar os parâmetros em cada algoritmo.</font><br><br>

#### K-Means

In [ ]:
labels_k = { 1: np.zeros((20,1)), 10: np.zeros((20,1)), 100: np.zeros((20,1)) }
for each_case in [1,10,100]:
    km = cluster.KMeans(n_clusters=5, random_state=0, max_iter=each_case).fit(x)
    labels_k[each_case] = km.labels_.reshape(-1,1)

In [ ]:
results = np.append(labels_k[1], np.append(labels_k[10], labels_k[100], axis=1), axis=1)
results = pd.DataFrame(results, columns=labels_k.keys())

In [ ]:
df = np.array([x for x in range(1,21)]).reshape(-20,1)
df = np.append(results.to_numpy(), df, axis=1)
df = pd.DataFrame(df, columns=['N_Iterations = 1', 'N_Iterations = 10', 'N_Iterations = 100', 'Teams'])

# Max_iterations = 1
df.plot.scatter(x='Teams',y='N_Iterations = 1',figsize=(10,5), title='Groups for Max Iterations = 1 vs Teams', c='g', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['N_Iterations = 1'])

# Max_iterations = 10
df.plot.scatter(x='Teams',y='N_Iterations = 10',figsize=(10,5), title='Groups for Max Iterations = 10 vs Teams', c='g', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['N_Iterations = 10'])

# Max_iterations = 100
df.plot.scatter(x='Teams',y='N_Iterations = 100',figsize=(10,5), title='Groups for Max Iterations = 100 vs Teams', c='g', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['N_Iterations = 100'])

<font size="3">&emsp;&emsp;Independente do número de iterações, os grupos continuam os mesmos, com os mesmos *labels*. Vamos normalizar os dados e comparar os resultados.</font><br><br>

#### Normalization

In [ ]:
normalized_premier_dataframe = normalize_df(premier_dataframe)

In [ ]:
x_norm = normalized_premier_dataframe.to_numpy()

labels_k_norm = { 1: np.zeros((20,1)), 10: np.zeros((20,1)), 100: np.zeros((20,1)) }
for each_case in [1,10,100]:
    km = cluster.KMeans(n_clusters=5, random_state=0, max_iter=each_case).fit(x_norm)
    labels_k_norm[each_case] = km.labels_.reshape(-1,1)

In [ ]:
results_norm = np.append(labels_k_norm[1], np.append(labels_k_norm[10], labels_k_norm[100], axis=1), axis=1)
results_norm = pd.DataFrame(results_norm, columns=labels_k_norm.keys())

In [ ]:
norm_df = np.array([x for x in range(1,21)]).reshape(-20,1)
norm_df = np.append(results_norm.to_numpy(), norm_df, axis=1)
norm_df = pd.DataFrame(norm_df, columns=['N_Iterations = 1', 'N_Iterations = 10', 'N_Iterations = 100', 'Teams'])

# Max_iterations = 1
norm_df.plot.scatter(x='Teams',y='N_Iterations = 1',figsize=(10,5), title='Groups for Max Iterations = 1 vs Teams', c='r', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['N_Iterations = 1'])

# Max_iterations = 10
norm_df.plot.scatter(x='Teams',y='N_Iterations = 10',figsize=(10,5), title='Groups for Max Iterations = 10 vs Teams', c='r', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['N_Iterations = 10'])

# Max_iterations = 100
norm_df.plot.scatter(x='Teams',y='N_Iterations = 100',figsize=(10,5), title='Groups for Max Iterations = 100 vs Teams', c='r', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['N_Iterations = 100'])

In [ ]:
def plot_results(df, n_df, key, x_label, y_label, color_1, color_2):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(111,xlabel=x_label, ylabel=y_label)
    ax1.scatter(df['Teams'],df[key],c=color_1,marker="s", label='Actual Values')
    ax1.scatter(n_df['Teams'],n_df[key],c=color_2,marker="o", label='Normalized Values')
    plt.legend(loc='best');
    rand_var = plt.xticks(n_df['Teams'])
    rand_var = plt.yticks(n_df[key])
    plt.title(f"{key} vs Teams")

    plt.show()

In [ ]:
# Comparing results
plot_results(df, norm_df, 'N_Iterations = 1', 'Teams', 'Groups','r','g')
plot_results(df, norm_df, 'N_Iterations = 10', 'Teams', 'Groups', 'r', 'g')
plot_results(df, norm_df, 'N_Iterations = 100', 'Teams', 'Groups', 'r', 'g')

<font size="3">&emsp;&emsp;Quanto maior o número de iterações maior a chance do algoritmo convergir, portanto vamos analisar o resultado do agrupamento normalizado vs valores reais quando o número de iterações é 100. No caso dos valores reais, o primeiro time tem um grupo separado só para ele, provavelmente porque o algoritmo deu maior peso para atributos relacionados a dinheiro, por exemplo, em finance_team_market onde o primeiro time possui valor de 1Bi, onde o resto dos valores possui valor de 836Mi ou menos. Porém, considerando os outros atributos, por exemplo, do segundo time com o primeiro time, temos que eles são parecidos, dado que a diferença entre seus valores não é muito grande. Outro caso que também pode ser considerado é que os times 3,4,5 e 6 poderíam estar juntos, principalmente 4,5 e 6, o que não é retratado nos valores reais, mas se aproxima do resultado desejado com os dados normalizados. Logo, o melhor resultado parece ser quando os dados estão normalizados. Porém, vamos usar uma métrica chamada Silhouette Score para avaliar os resultados.</font><br><br>

In [ ]:
pure_score = metrics.silhouette_score(x, df['N_Iterations = 100'])
norm_score = metrics.silhouette_score(x_norm, norm_df['N_Iterations = 100'])
print(f'Com os dados inalterados: {pure_score}\nCom os dados normalizados: {norm_score}')

<font size="3">&emsp;&emsp;Segundo o Silhouette Score, temos que o resultado é exatamente o oposto do que pensávamos. O melhor resultado para essas condições é quando os dados não estão normalizados.</font><br><br>

#### AgglomerativeClustering (Hierarchical)

In [ ]:
labels_a = {"ward": np.zeros((20,1)), "complete": np.zeros((20,1)), "average": np.zeros((20,1)), "single": np.zeros((20,1))}
for each_case in ["ward", "complete", "average", "single"]:
    ag = cluster.AgglomerativeClustering(n_clusters=5, linkage=each_case).fit(x)
    labels_a[each_case] = ag.labels_.reshape(-1,1)

In [ ]:
results = np.append(labels_a["ward"], np.append(labels_a["complete"], np.append(labels_a["average"], labels_a["single"], axis=1), axis=1), axis=1)
results = pd.DataFrame(results, columns=labels_a.keys())
results

In [ ]:
df = np.array([x for x in range(1,21)]).reshape(-20,1)
df = np.append(results.to_numpy(), df, axis=1)
df = pd.DataFrame(df, columns=['Ward (Groups)', 'Complete (Groups)', 'Average (Groups)', 'Single (Groups)', 'Teams'])

# Linkage = ward
df.plot.scatter(x='Teams',y='Ward (Groups)',figsize=(10,5), title='Groups for linkage = ward vs Teams', c='b', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['Ward (Groups)'])

# Linkage = complete
df.plot.scatter(x='Teams',y='Complete (Groups)',figsize=(10,5), title='Groups for linkage = complete vs Teams', c='b', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['Complete (Groups)'])

# Linkage = average
df.plot.scatter(x='Teams',y='Average (Groups)',figsize=(10,5), title='Groups for linkage = average vs Teams', c='b', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['Average (Groups)'])

# Linkage = single
df.plot.scatter(x='Teams',y='Single (Groups)',figsize=(10,5), title='Groups for linkage = single vs Teams', c='b', marker='o')
rand_var = plt.xticks(df['Teams'])
rand_var = plt.yticks(df['Single (Groups)'])

<font size="3">&emsp;&emsp;Nos casos em que o linkage é *Complete* ou *Average*, os resultados são os mesmos, apenas trocando a ordem dos *clusters* 4,2 e 1. Já os métodos *ward* e *single* são singulares, gerando resultados diferentes um do outro e dos outros dois métodos simultaneamente. Vamos normalizar e ver a diferença entre os resultados.</font><br><br>

#### Normalization

In [ ]:
labels_a_norm = {"ward": np.zeros((20,1)), "complete": np.zeros((20,1)), "average": np.zeros((20,1)), "single": np.zeros((20,1))}
for each_case in ["ward", "complete", "average", "single"]:
    ag = cluster.AgglomerativeClustering(n_clusters=5, linkage=each_case).fit(x_norm)
    labels_a_norm[each_case] = ag.labels_.reshape(-1,1)

In [ ]:
results_norm = np.append(labels_a_norm['ward'], np.append(labels_a_norm['complete'], np.append(labels_a_norm['average'], labels_a_norm['single'], axis=1), axis=1), axis=1)
results_norm = pd.DataFrame(results_norm, columns=labels_a_norm.keys())

In [ ]:
norm_df = np.array([x for x in range(1,21)]).reshape(-20,1)
norm_df = np.append(results_norm.to_numpy(), norm_df, axis=1)
norm_df = pd.DataFrame(norm_df, columns=['Ward (Groups)', 'Complete (Groups)', 'Average (Groups)', 'Single (Groups)', 'Teams'])

# Linkage = ward
norm_df.plot.scatter(x='Teams',y='Ward (Groups)',figsize=(10,5), title='Groups for linkage = ward vs Teams', c='y', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['Ward (Groups)'])

# Linkage = complete
norm_df.plot.scatter(x='Teams',y='Complete (Groups)',figsize=(10,5), title='Groups for linkage = complete vs Teams', c='y', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['Complete (Groups)'])

# Linkage = average
norm_df.plot.scatter(x='Teams',y='Average (Groups)',figsize=(10,5), title='Groups for linkage = average vs Teams', c='y', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['Average (Groups)'])

# Linkage = single
norm_df.plot.scatter(x='Teams',y='Single (Groups)',figsize=(10,5), title='Groups for linkage = single vs Teams', c='y', marker='s')
rand_var = plt.xticks(norm_df['Teams'])
rand_var = plt.yticks(norm_df['Single (Groups)'])

In [ ]:
# Comparing results
plot_results(df, norm_df, 'Ward (Groups)', 'Teams', 'Groups','b','y')
plot_results(df, norm_df, 'Complete (Groups)', 'Teams', 'Groups', 'b', 'y')
plot_results(df, norm_df, 'Average (Groups)', 'Teams', 'Groups', 'b', 'y')
plot_results(df, norm_df, 'Single (Groups)', 'Teams', 'Groups', 'b', 'y')

<font size="3">&emsp;&emsp;Podemos ver que os resultados são bem diferentes dos dados sem normalização e com normalização. Provavelmente o resultado não normalizado será denovo melhor que o normalizado, mas para ter certeza comparemos o Silhoutte score para cada caso.</font><br><br>

In [ ]:
# Ward
pure_score = metrics.silhouette_score(x, df['Ward (Groups)'])
norm_score = metrics.silhouette_score(x_norm, norm_df['Ward (Groups)'])
print(f'Dados inalterados vs normalizados (ward): {pure_score} / {norm_score}')

# Complete average single
pure_score = metrics.silhouette_score(x, df['Complete (Groups)'])
norm_score = metrics.silhouette_score(x_norm, norm_df['Complete (Groups)'])
print(f'Dados inalterados vs normalizados (complete): {pure_score} / {norm_score}')

# Average
pure_score = metrics.silhouette_score(x, df['Average (Groups)'])
norm_score = metrics.silhouette_score(x_norm, norm_df['Average (Groups)'])
print(f'Dados inalterados vs normalizados (average): {pure_score} / {norm_score}')

# Single
pure_score = metrics.silhouette_score(x, df['Single (Groups)'])
norm_score = metrics.silhouette_score(x_norm, norm_df['Single (Groups)'])
print(f'Dados inalterados vs normalizados (single): {pure_score} / {norm_score}')

<font size="3">&emsp;&emsp;Como esperado, o melhor resultado foi no caso não normalizado. Os melhores métodos linkage para os casos reais e normalizados foram complete e average, também como esperado ja que obtiveram os mesmos resultados.</font><br><br>

### Concluindo
<font size="3">&emsp;&emsp;Comparando os melhores casos dos dois algoritmos, baseado no Silhouette Score, temos:<br>&emsp;&emsp;&emsp;&emsp;K-Means (não normalizado com 100 interações) = 0.5068576386751202<br>&emsp;&emsp;&emsp;&emsp;Agglomerative (não normalizado com linkage complete) = 0.5068576386751202<br><br>&emsp;&emsp;O resultado é o mesmo, apenas trocando de lugar o label de dois grupos. O Silhouette Score varia de -1 a 1, sendo os objetos mal agrupados e bem agrupados respectivamente. O valor ter dado 0.5068576386751202, indica que o agrupamento foi relativamente bom e talvez pudesse ter sido melhor caso os dados tivessem um tratamento melhor, ou os parâmetros fossem diferentes. Contudo, o resultado foi satisfatório nos dois algoritmos, dada a aproximação superficial que foi aplicada para esse projeto.</font><br><br>